In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import itertools

from itertools import islice
from collections import Counter, defaultdict

from itertools import chain
import ast


import statistics
import numpy as np

In [8]:
# pickle dfs
ipc = ["H04W", "H01L", "G06F", "G01N", "E21B", "A61B"]

for i in ipc:
    toEval = pd.read_csv(f'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/toEval/2016_{i}_patents_toEval.csv')
    toEval.to_pickle(f'pickledDF/2016_{i}_patents_toEval.pkl')
    KS = pd.read_csv(f'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/KS/2016_1115_{i}_KS_raw.csv')
    KS.to_pickle(f'pickledDF/2016_1115_{i}_KS_raw.pkl')

In [19]:
KS_sec_ipc = [[] for i in ipc]

for idx,i in enumerate(tqdm(ipc)):
    KS_sec_ipc[idx] = list(pd.Series([ast.literal_eval(i) for i in pd.read_pickle(f'pickledDF/2016_1115_{i}_KS_raw.pkl').sec_ipc]))

100%|██████████| 6/6 [00:16<00:00,  2.75s/it]


In [24]:
len(KS_sec_ipc[2])

173662

In [91]:
len_KS_sec = [[] for i in ipc]
for idx, i in enumerate(tqdm(ipc)):
    len_KS_sec[idx] = [len(x) for x in KS_sec_ipc[idx] if len(x)]
    df_counter = pd.DataFrame(Counter(len_KS_sec[idx]).items(), columns=['n', 'Fréquence'])
    df_counter = df_counter.sort_values(by='n').reset_index(drop=True)
    print(f"{i}: ")
    print(f"Nombre de brevets avec n IPCs (incluant le principal) dans la KS de {i}: ")
    print(df_counter[:3])
    print("Juste 1 IPC: ", round(Counter(len_KS_sec[idx])[1]/len(KS_sec_ipc[idx])*100,1), "%")
    print("Nb IPCs moyens/brevet: ", round(np.mean(len_KS_sec[idx]), 3))
    print("Nb brevets dans KS:", len(len_KS_sec[idx]))
    print("")


100%|██████████| 6/6 [00:00<00:00, 32.52it/s]

H04W: 
Nombre de brevets avec n IPCs (incluant le principal) dans la KS de H04W: 
   n  Fréquence
0  1      30772
1  2      15107
2  3       7036
Juste 1 IPC:  53.8 %
Nb IPCs moyens:  1.768
Nb brevets dans KS: 57161

H01L: 
Nombre de brevets avec n IPCs (incluant le principal) dans la KS de H01L: 
   n  Fréquence
0  1      36980
1  2      32479
2  3      13842
Juste 1 IPC:  37.3 %
Nb IPCs moyens:  2.294
Nb brevets dans KS: 99087

G06F: 
Nombre de brevets avec n IPCs (incluant le principal) dans la KS de G06F: 
   n  Fréquence
0  1     122799
1  2      34367
2  3      10747
Juste 1 IPC:  70.7 %
Nb IPCs moyens:  1.44
Nb brevets dans KS: 173662

G01N: 
Nombre de brevets avec n IPCs (incluant le principal) dans la KS de G01N: 
   n  Fréquence
0  1      15655
1  2       7139
2  3       3222
Juste 1 IPC:  54.6 %
Nb IPCs moyens:  1.856
Nb brevets dans KS: 28677

E21B: 
Nombre de brevets avec n IPCs (incluant le principal) dans la KS de E21B: 
   n  Fréquence
0  1       5831
1  2       3797
2 

In [69]:
count_full=pd.DataFrame()
for idx, i in enumerate(ipc):
    df = pd.DataFrame({"KS_sec_ipc": KS_sec_ipc[idx]})
    # Step 1: Explode the lists into separate rows
    exploded_df = df.explode('KS_sec_ipc')

    # Step 2: Count occurrences of each unique code
    count_series = exploded_df['KS_sec_ipc'].value_counts()

    # Step 3: Convert to DataFrame for better readability
    count_df = count_series.reset_index()
    count_df.columns = [f'{i}', 'Count']  # Rename columns
    count_full = pd.concat([count_full, count_df], axis=1)

# Display the count DataFrame
print("Nb IPCs")
count_full[:10]

Nb IPCs


,H04W,Count,H01L,Count,G06F,Count,G01N,Count,E21B,Count,A61B,Count
0,H04W7204,11366.0,H01L2966,7696,G06F1730,29339.0,G01N2164,1976.0,E21B4700,715.0,A61B500,8173.0
1,H04W400,4983.0,H01L2978,7307,G06F1516,10289.0,G01N3368,1968.0,E21B3312,700.0,A61B600,2675.0
2,H04W402,3768.0,H01L2102,6441,G06F3041,9025.0,G01N3350,1306.0,E21B4326,655.0,A61B1900,2541.0
3,H04W2400,3457.0,H01L21768,4752,G06F1900,7210.0,G01N3353,1041.0,E21B3406,630.0,A61B1814,1927.0
4,H04W7602,3198.0,H01L3118,4640,G06F30484,6279.0,G01N3300,994.0,E21B4324,524.0,A61B800,1834.0
5,H04W5202,3015.0,H01L2300,4147,G06F1750,6181.0,G01N33574,986.0,E21B4316,505.0,A61B1770,1777.0
6,H04W3600,2952.0,H01L2906,3976,G06F301,5502.0,G01N33543,864.0,E21B4312,500.0,A61B100,1730.0
7,H04W1206,1776.0,H01L5100,3493,G06F944,4963.0,C12Q168,728.0,E21B4100,491.0,A61B1700,1576.0
8,H04W7208,1733.0,H01L29786,3096,G06F306,4696.0,G01N33566,707.0,E21B4300,490.0,A61B808,1491.0
9,H04W2402,1648.0,H01L5152,3063,G06F1700,3984.0,G01N33569,667.0,E21B4712,473.0,A61B511,1468.0


In [58]:
from itertools import combinations
def count_combinations(series, combo_size):
    # List to hold combinations
    all_combinations = []
    
    # Iterate over each row in the series
    for sublist in series:
        # Generate combinations of the specified size
        if len(sublist) >= combo_size:  # Only consider lists with enough elements
            all_combinations.extend(combinations(sorted(sublist), combo_size))
    
    # Count occurrences of each combination
    combination_counts = Counter(all_combinations)
    
    return combination_counts

In [87]:
pc2 = pd.DataFrame()
pc3 = pd.DataFrame()

for idx, i in enumerate(tqdm(ipc)):
    pair_counts2 = count_combinations([sublist for sublist in KS_sec_ipc[idx] if len(sublist)>0], 2)
    pair_counts_df2 = pd.DataFrame(pair_counts2.items(), columns=['Combination of 2', 'Count']).sort_values(by='Count', ascending=False)
    pair_counts3 = count_combinations([sublist for sublist in KS_sec_ipc[idx] if len(sublist)>0], 3)
    pair_counts_df3 = pd.DataFrame(pair_counts3.items(), columns=['Combination of 3', 'Count']).sort_values(by='Count', ascending=False)
    pc2 = pd.concat([pc2, pair_counts_df2.reset_index(drop=True)[:10]], axis=1)
    pc3 = pd.concat([pc3, pair_counts_df3.reset_index(drop=True)[:10]], axis=1)

100%|██████████| 6/6 [00:01<00:00,  4.15it/s]


In [89]:
pc2

,Combination of 2,Count,Combination of 2,Count,Combination of 2,Count,Combination of 2,Count,Combination of 2,Count,Combination of 2,Count
0,"(H04L500, H04W7204)",970,"(H01L2966, H01L2978)",2656,"(G06F30482, G06F30484)",1494,"(C12Q168, G01N33574)",202,"(E21B4700, E21B4712)",87,"(A61B500, A61B50205)",1053
1,"(H04W400, H04W7602)",499,"(H01L2102, H01L2966)",1415,"(G06F30481, G06F30484)",1435,"(C12Q168, G01N3368)",115,"(E21B3400, E21B3406)",84,"(A61B500, A61B511)",1031
2,"(H04W7204, H04W7208)",472,"(H01L5152, H01L5156)",1291,"(G06F3041, G06F3044)",1425,"(G01N2188, G01N2195)",105,"(E21B3312, E21B3406)",79,"(A61B500, A61B5145)",812
3,"(H04W7204, H04W7602)",449,"(H01L2906, H01L2978)",1282,"(G06F30484, G06F30488)",1371,"(C12Q168, G01N2164)",99,"(E21B4316, E21B4324)",76,"(A61B800, A61B808)",812
4,"(H04W400, H04W7204)",405,"(H01L2906, H01L2966)",1156,"(G06F1730, H04L2908)",1044,"(G01N2164, G01N33566)",96,"(E21B4326, E21B43267)",75,"(A61B500, A61B51455)",714
5,"(H04W2400, H04W7204)",345,"(H01L2732, H01L5152)",1093,"(G06F3041, G06F30488)",810,"(G01N2164, G01N2176)",88,"(E21B4700, E21B4706)",73,"(A61B500, A61B504)",655
6,"(H04W5202, H04W7204)",344,"(H01L2966, H01L29786)",966,"(G06F30481, G06F30488)",725,"(G01N3350, G01N3368)",86,"(E21B4706, E21B4712)",67,"(A61B600, A61B603)",618
7,"(H04W7204, H04W7408)",315,"(H01L2732, H01L5156)",937,"(G06F1730, G06F30484)",697,"(G01N2164, G01N3353)",80,"(E21B4316, E21B4326)",64,"(A61B500, A61B501)",532
8,"(H04W402, H04W6400)",304,"(H01L21336, H01L2978)",874,"(G06F301, G06F3041)",675,"(C12Q168, G01N3350)",76,"(E21B3312, E21B4326)",57,"(A61B500, A61B5024)",526
9,"(H04L2906, H04W1206)",303,"(H01L29423, H01L2966)",873,"(G06F30481, G06F30482)",649,"(C12Q168, G01N33566)",69,"(E21B3312, E21B3313)",53,"(A61B50205, A61B511)",425


In [90]:
pc3

,Combination of 3,Count,Combination of 3,Count,Combination of 3,Count,Combination of 3,Count,Combination of 3,Count,Combination of 3,Count
0,"(H04W400, H04W7602, H04W800)",86,"(H01L2906, H01L2966, H01L2978)",643,"(G06F30481, G06F30484, G06F30488)",437,"(G01N2164, G01N2176, G01N33566)",22,"(C09K880, E21B4326, E21B43267)",20,"(A61B500, A61B50205, A61B511)",363
1,"(H04L118, H04L500, H04W7204)",77,"(H01L2732, H01L5152, H01L5156)",498,"(G06F30482, G06F30484, G06F30488)",408,"(C12Q168, G01N2164, G01N33566)",21,"(E21B4316, E21B4324, E21B4326)",17,"(A61B800, A61B808, A61B814)",321
2,"(H04L500, H04L514, H04W7204)",63,"(H01L2102, H01L2966, H01L2978)",443,"(G06F30481, G06F30482, G06F30484)",370,"(C12Q168, C40B3004, G01N33574)",20,"(B24D1800, E21B1055, E21B10567)",15,"(A61B500, A61B50205, A61B51455)",266
3,"(H04L500, H04W7204, H04W7208)",61,"(H01L2910, H01L2966, H01L2978)",387,"(G06F1730, G06F30482, G06F30484)",273,"(C12Q168, G01N3350, G01N33574)",18,"(E21B33124, E21B4314, E21B4326)",14,"(A61B500, A61B50205, A61B5145)",201
4,"(H04L2726, H04L500, H04W7204)",53,"(H01L2156, H01L2300, H01L2331)",381,"(G06F3041, G06F30484, G06F30488)",271,"(C12Q168, G01N3350, G01N3368)",18,"(E21B4700, E21B4706, E21B4712)",13,"(A61B500, A61B5145, A61B51455)",193
5,"(H04L2906, H04W1206, H04W1208)",51,"(H01L29423, H01L2966, H01L2978)",361,"(G06F30481, G06F30482, G06F30488)",214,"(C40B3004, G01N2164, G01N33566)",17,"(E21B1720, E21B1908, E21B1922)",10,"(A61B500, A61B50205, A61B50402)",154
6,"(H04L500, H04W7204, H04W7212)",49,"(H01L2906, H01L2910, H01L2978)",349,"(G06F301, G06F30484, G06F30488)",138,"(G01N3500, G01N3502, G01N3510)",16,"(E21B2108, E21B2110, E21B4706)",10,"(A61B500, A61B501, A61B511)",147
7,"(H04L2906, H04W1204, H04W1206)",47,"(H01L2102, H01L2906, H01L2966)",304,"(G06F1730, H04L2906, H04L2908)",137,"(C12Q168, G01N2164, G01N33574)",16,"(E21B4400, E21B4706, E21B4712)",10,"(A61B500, A61B501, A61B5145)",132
8,"(H04W7204, H04W7602, H04W800)",45,"(H01L2908, H01L2966, H01L2978)",281,"(G06F301, G06F3041, G06F30488)",124,"(C07K1618, C12Q168, G01N33574)",16,"(E21B4326, E21B4706, E21B4712)",9,"(A61B500, A61B511, A61B5145)",132
9,"(H04L500, H04W2408, H04W7204)",44,"(H01L2712, H01L2966, H01L29786)",280,"(G06F3041, G06F30481, G06F30488)",119,"(A61K39395, A61K4506, G01N3368)",15,"(E21B4316, E21B4326, E21B43267)",9,"(A61B500, A61B501, A61B50205)",131
